In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 20 11:12:31 2025

@author: aglabassi
"""



import os
import torch
import pickle
from result_processing_methods  import collect_compute_mean,\
 plot_losses_with_styles, plot_transition_heatmap, plot_results_sensitivity
from run_preconditioned_subgradient_experiments import test_methods
from itertools import product

!sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended
#!git clone https://<YOUR_TOKEN>@github.com/aglabassi/preconditioned_composite_optimization.git

torch.set_default_dtype(torch.float64)

# Set device to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

def save(obj, filename):
    """Saves an object to a file using pickle."""
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
    print(f"Object saved to {filename}")

def load(filename):
    """Loads an object from a file using pickle."""
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    print(f"Object loaded from {filename}")
    return obj






Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cm-super-minimal dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern
  fonts-noto-mono fonts-texgyre fonts-urw-base35 ghostscript
  libapache-pom-java libcommons-logging-java libcommons-parent-java
  libfontbox-java libfontenc1 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0 libsynctex2
  libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13 lmodern
  pfb2t1c2pfb poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils
  tex-common tex-gyre texlive-base texlive-binaries texlive-fonts-recommended
  texlive-latex-base texlive-pictures texlive-plain-generic tipa
  xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf ghostscript-x
  libavalon-fram

# **Experiment 1: Exact observations with Polyak Stepsizes**

In [14]:
# Parmeter definitions.
n1 = 100
n2 = 100
n3 = 100

loss_ord = 2 #Set to 2 for l2 norm squared, 0.5 for L2 norm not squared (better precision in PyTorch), 1 for L1 norm.


r_true = 2
m = n1 * r_true * 20
symmetric = True      # Set True for symmetric.
tensor = False

if symmetric:
  n2 = n1
  n3 = n1

identity = False #Set to true for factorization. Otherwise performs sensing on Gaussian measurements
base_dir = os.path.join(os.getcwd(), 'experiment_results/polyak')
os.makedirs(base_dir, exist_ok=True)
initial_relative_error = 10**-2
n_iter = 500


# experiment_setups: (overparameterization, condition number)
experiment_setups = [(2,1), (5,1), (2,100), (5, 100)]
methods = ['Levenberg-Marquardt (ours)']

if loss_ord == 2:
  to_add = ['Gradient descent']

  if not tensor and symmetric:
      to_add += ['Precond. gradient', 'Scaled gradient($\lambda=10^{-8}$)' ]
  elif not tensor:
      to_add += ['Scaled gradient($\lambda=10^{-8}$)' ]
elif loss_ord == 1:
  to_add = ['Subgradient descent']
  if not tensor:
      to_add += [ 'OPSA($\lambda=10^{-8}$)' ] if not symmetric else []
elif loss_ord == 0.5:
  to_add = ['Subgradient descent']


methods = methods + to_add
methods_test = methods

# Run the methods.
test_methods(methods_test, experiment_setups, n1, n2, n3, r_true, m, identity, device,
          n_iter, base_dir, loss_ord, initial_relative_error, symmetric,
          tensor=tensor)

# Compute and plot errors.
errs, stds = collect_compute_mean(experiment_setups, loss_ord, r_true, False, methods,
                                f'{"tensor" if tensor else "matrix"}{"sym" if symmetric else ""}', base_dir
                                  )
plot_losses_with_styles(errs, stds, r_true, loss_ord, base_dir,
                        (('Symmetric ' if symmetric else 'Asymmetric ') +
                        ('Tensor' if tensor else 'Matrix')), 1)






Experiment Setup: r = 2, kappa = 1

--------------------------------------------------------------------------------
Starting method: Levenberg-Marquardt (ours)
--------------------------------------------------------------------------------
  Levenberg-Marquardt (ours)   | Iteration: 000 | Relative Error: 1.005e-02
  Levenberg-Marquardt (ours)   | Iteration: 020 | Relative Error: 6.683e-04
  Levenberg-Marquardt (ours)   | Iteration: 040 | Relative Error: 5.766e-05
  Levenberg-Marquardt (ours)   | Iteration: 060 | Relative Error: 5.545e-06
  Levenberg-Marquardt (ours)   | Iteration: 080 | Relative Error: 5.692e-07
  Levenberg-Marquardt (ours)   | Iteration: 100 | Relative Error: 6.108e-08
  Levenberg-Marquardt (ours)   | Iteration: 120 | Relative Error: 6.758e-09
  Levenberg-Marquardt (ours)   | Iteration: 140 | Relative Error: 7.635e-10
  Levenberg-Marquardt (ours)   | Iteration: 160 | Relative Error: 8.752e-11
  Levenberg-Marquardt (ours)   | Iteration: 180 | Relative Error: 1.014e-1

RuntimeError: Failed to process string with tex because latex could not be found

Error in callback <function _draw_all_if_interactive at 0x7f5665fe0cc0> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 3600x2700 with 1 Axes>

# **Experiment 2: Geometrically Decaying Hyperparemeters: Sensitivity Analysis**

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 20 11:12:31 2025

@author: aglabassi
"""








run = True
tensor = False
symmetric = True

n=n1=n2=n3=10 if tensor  else 40
r_true = 2
m = n * r_true * 20
r = 5
initial_relative_error = 10**-3
rel_error_exp = 8
rel_epsilon_stop = 10**(-1*rel_error_exp)
tests = [ (2,1), (2,100), (5,100)]
problems = [ ' ' ] #ignore this

methods = [ 'Subgradient descent', 'Gauss-Newton','Levenberg–Marquardt (ours)']
corruption_levels = [0]
qs = [ 0.95, 0.96, 0.97  ]
lambdas = [ 10**-5 ]
gammas = [10**i for i in range(4,0, -1)] + [10**-i for i in range(0,10)]
K  = 1000
n_trial = 50
np.random.seed(42)

base_dir = os.path.join(os.getcwd(), 'experiment_results/phase_transition')
os.makedirs(os.path.dirname(base_dir), exist_ok=True)


if run:
    for corr_level, (r_test, c), q in product(corruption_levels, tests, qs):

        to_be_plotted = dict( (problem, dict( (method,np.zeros((len(lambdas), len(gammas)), dtype=object)) for method in methods)) for problem in problems )

        for problem in problems:
            for i, j in product(range(len(lambdas)), range(len(gammas))):
                last_indexes = dict(  (method, [] ) for method in methods)

                for _ in range(n_trial):
                    # Generate losses depending on the problem
                    outputs = run_methods(methods, [(r_test, c)], n1, n2, n3, r_true, m, False, device,
                            K, False, base_dir, 1, initial_relative_error, symmetric,
                            tensor=tensor, corr_level=corr_level, q=q, lambda_=lambdas[i], gamma=gammas[j], geom_decay=True)


                    for method in methods:
                        losses = outputs[method]
                        # Find the first iteration index where losses < rel_epsilon_stop
                        idx = np.where(np.array(losses) < rel_epsilon_stop)[0]
                        last_index = idx[0] if len(idx) > 0 else K
                        last_indexes[method].append(last_index)

                for method in methods:
                    median_last_index = np.median(last_indexes[method])
                    shaded = np.percentile(last_indexes[method], [5, 95])
                    to_be_plotted[problem][method][i, j] = (median_last_index, tuple(shaded))
            save_path = os.path.join(base_dir, f'to_be_plotted_{problem}_{corr_level}_{r_test}_{c}_{q}.pkl')
            save(to_be_plotted[problem], save_path)

# Call the plotting function
font_size=30
for corr_level, (r_test, c), q in product(corruption_levels, tests, qs):
    for problem, method  in product(problems, methods):
        to_be_plotted = load(os.path.join(base_dir, f'to_be_plotted_{problem}_{corr_level}_{r_test}_{c}_{q}.pkl'))
        plot_results_sensitivity(to_be_plotted, corr_level, q, r_test, c, gammas, lambdas, font_size, rel_error_exp, problem, base_dir)



# **Experiment 3: Sensing with Outliers**

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 20 11:12:31 2025

@author: aglabassi
"""




run = True
tensor=False
symmetric = True
n1 = n2 = n3 = 10 if tensor else 40

loss_ord = 1

trials = 50
n_trial_div_n_cpu = 1

T = 1000


methods = ['Subgradient descent','Gauss-Newton', 'Levenberg–Marquardt (ours)']
methods_test = methods
methods_all = methods

init_radius_ratio =10**-2
end_ratio = 10**-10
keys_all = [(5,100)] #keep one
keys_test = keys_all

d_trials = [ i*2*n for i in range(1,20)]
cor_interval= 0.025
corr_ranges = [ [l, l+cor_interval]  for l in np.arange(0, 0.5, cor_interval)]

base_dir = base_dir = os.path.join(os.getcwd(), 'experiment_results/phase_transition')
save_path = os.path.join(base_dir,f'{keys_test[0]}_{problem}.pkl' )
os.makedirs(os.path.dirname(base_dir), exist_ok=True)

if run:
    success_matrixes = dict( (method, np.zeros((len(d_trials), len(corr_ranges)) )) for method in methods)
    for i,d_trial in enumerate(d_trials):
        for j,corr_range in enumerate(corr_ranges):

            success_counters = dict((method, 0) for method in methods)

            for _ in range(trials):
                corr_factor = (corr_range[1] -corr_range[0]) *np.random.rand() + corr_range[0]
                d = d_trial

                outputs = run_methods(methods_test, keys_all, n1, n2, n3, r_true, m, False, device,
                            n_iter, spectral_init, base_dir, loss_ord, init_radius_ratio, symmetric,
                            tensor=tensor, corr_level=corr_factor, geom_decay=True)




                for method in methods:
                    if outputs[method][-1] <= end_ratio:
                        success_counters[method] +=1

            for method in methods:
                success_matrixes[method][i,j] = success_counters[method]/trials

    # Save success matrix

    save(success_matrixes, save_path)

# Load success matrix for verification or further processing
loaded_success_matrixes = load(save_path)

plot_transition_heatmap(
    success_matrixes=loaded_success_matrixes,
    d_trials=d_trials,
    n=n,
    base_dir=base_dir,
    keys = keys_test[0],
    problem=problem,
)
